<a href="https://colab.research.google.com/github/Andres8bit/Machine-Learning/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To large to Train on Free GPU access: 
#           One Epoch took longer than 1 hour
from keras.models import Sequential
from keras.regularizers import L2
from keras.optimizers import SGD
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Activation, MaxPool2D,BatchNormalization,Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os
import sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Dataset Download:
base_path = "/content/drive/MyDrive/bird Species.zip (Unzipped Files)"
train_path = os.path.join(base_path,'train')
test_path = os.path.join(base_path,'test')
validation_path = os.path.join(base_path,'valid')

In [ ]:
# Load Datasets to Tensorflow Dataset:
training_set = image_dataset_from_directory( directory=train_path, labels='inferred', label_mode='categorical', class_names=None,
    color_mode='rgb', batch_size=32, image_size=(227, 227),
    shuffle=True)
test_set = image_dataset_from_directory(test_path,labels='inferred',label_mode='categorical',class_names=None,color_mode='rgb',batch_size=32,shuffle=True,image_size=(227,227))
valid_set = image_dataset_from_directory(validation_path,labels='inferred',label_mode='categorical',class_names=None,color_mode='rgb',shuffle=True,batch_size=32,image_size=(227,227))

Found 39364 files belonging to 275 classes.
Found 1375 files belonging to 275 classes.
Found 1375 files belonging to 275 classes.


In [ ]:
training_set.class_names

['AFRICAN CROWNED CRANE',
 'AFRICAN FIREFINCH',
 'ALBATROSS',
 'ALEXANDRINE PARAKEET',
 'AMERICAN AVOCET',
 'AMERICAN BITTERN',
 'AMERICAN COOT',
 'AMERICAN GOLDFINCH',
 'AMERICAN KESTREL',
 'AMERICAN PIPIT',
 'AMERICAN REDSTART',
 'ANHINGA',
 'ANNAS HUMMINGBIRD',
 'ANTBIRD',
 'ARARIPE MANAKIN',
 'ASIAN CRESTED IBIS',
 'BALD EAGLE',
 'BALI STARLING',
 'BALTIMORE ORIOLE',
 'BANANAQUIT',
 'BANDED BROADBILL',
 'BAR-TAILED GODWIT',
 'BARN OWL',
 'BARN SWALLOW',
 'BARRED PUFFBIRD',
 'BAY-BREASTED WARBLER',
 'BEARDED BARBET',
 'BEARDED REEDLING',
 'BELTED KINGFISHER',
 'BIRD OF PARADISE',
 'BLACK & YELLOW bROADBILL',
 'BLACK FRANCOLIN',
 'BLACK SKIMMER',
 'BLACK SWAN',
 'BLACK TAIL CRAKE',
 'BLACK THROATED BUSHTIT',
 'BLACK THROATED WARBLER',
 'BLACK VULTURE',
 'BLACK-CAPPED CHICKADEE',
 'BLACK-NECKED GREBE',
 'BLACK-THROATED SPARROW',
 'BLACKBURNIAM WARBLER',
 'BLUE GROUSE',
 'BLUE HERON',
 'BOBOLINK',
 'BORNEAN BRISTLEHEAD',
 'BORNEAN LEAFBIRD',
 'BROWN NOODY',
 'BROWN THRASHER',
 'BULWERS

In [ ]:
class_names = training_set.class_names

# plt.figure(figsize=(12,12))
# for images, labels in training_set.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")

In [ ]:
# Buffered Prefetching for better performance by overlaping model execution and data preprocessing:
AUTOTUNE = tf.data.AUTOTUNE

training_set = training_set.prefetch(buffer_size=AUTOTUNE)
test_set = test_set.prefetch(buffer_size=AUTOTUNE)
valid_set = valid_set.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Model Architecture:
model = Sequential()

# 1st Convolutional layer, input = (227,227,3):
model.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4), padding='valid', 
                 input_shape=(227,227,3)))
# Relu activation Layer:
model.add(Activation('relu'))
# Pooling Layer:
model.add(MaxPool2D(pool_size=(3,3),strides=(2,2)))
# BatchNormalization Layer:
model.add(BatchNormalization())

# 2nd Convolutional Layer:
model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding='same',
                 kernel_regularizer=L2(0.0005)))
# Relu activation Layer:
model.add(Activation('relu'))
# Pooling Layer:
model.add(MaxPool2D(pool_size=(3,3),strides=(2,2),padding='valid'))
# BatchNormalization Layer:
model.add(BatchNormalization())

# 3rd  Convolutional Layer:
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='same',
                 kernel_regularizer=L2(0.0005)))
# Relu Activation Layer:
model.add(Activation('relu'))
# BatchNormalization Layer:
model.add(BatchNormalization())

# 4th Convolutional Layer:
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='same', 
                 kernel_regularizer=L2(0.0005)))
# Activation Layer:
model.add(Activation('relu'))
# BathNormalization:
model.add(BatchNormalization())

# 5th Convolutional Layer:
model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',
                 kernel_regularizer=L2(0.0005)))
# Activation Layer:
model.add(Activation('relu'))
# BatchNormalization:
model.add(BatchNormalization())
# MaxPooling Layer:
model.add(MaxPool2D(pool_size=(3,3),strides=(2,2),padding='valid'))

# Flatten Convolution for Fully Connected Layers:
model.add(Flatten())

# 6th Dense Layer:
model.add(Dense(units=4096,activation='relu'))
# Dropout layer:
model.add(Dropout(0.5))

# 7th Dense Layer:
model.add(Dense(units=275,activation='softmax'))

model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0

In [ ]:
# Learing Rate Scheduler & SGD Optimizer:
reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',factor=np.sqrt(0.1))
optimizer = SGD(learning_rate=0.1,momentum=0.9)


In [ ]:
# Compile Model:
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
# Fit Model:
model.fit(training_set,batch_size=128,epochs=90,validation_data=valid_set,verbose=2,callbacks=[reduce_learning_rate])

Epoch 1/90
